In [58]:
import numpy as np
import la_forge.core as co
import pickle, json, copy
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
from enterprise import constants as const

from enterprise_extensions.frequentist.optimal_statistic import OptimalStatistic as OS
from enterprise_extensions.chromatic import chromatic
from enterprise_extensions import hypermodel

In [13]:
import la_forge.core as co
import la_forge.diagnostics as dg

In [15]:
import numpy as np

In [2]:
pkl = '/Users/hazboun/ipta/classic_psrs_de438.pkl'
with open(pkl,'rb')as fin:
    psrs = pickle.load(fin)

In [3]:
rm_idxs = []
for idx, p in enumerate(psrs):
    first = p.toas.min()
    last = p.toas.max()
    tspan = (last - first)/(365.25*24*3600)
    if tspan<3.0:
        rm_idxs.append(idx)
#     print(p.name, f'{tspan:2.1f}')

for idx in reversed(rm_idxs):
    del psrs[idx]

In [4]:
tmin = np.amin([p.toas.min() for p in psrs])
tmax = np.amax([p.toas.max() for p in psrs])
Tspan = tmax - tmin
Tspan/(365.25*24*3600)

30.190711132332524

In [5]:
# Red noise parameter priors
log10_A = parameter.Uniform(-20, -11)
gamma = parameter.Uniform(0, 7)

# DM Red noise parameter priors
dm_log10_A = parameter.Uniform(-20, -11)
dm_gamma = parameter.Uniform(0, 7)

# GW parameter priors
gw_log10_A = parameter.Uniform(-18, -14)('gw_log10_A')
gw_gamma = parameter.Constant(13./3)('gw_gamma')

# White noise parameter priors
efac = parameter.Constant()
equad = parameter.Constant()
ecorr = parameter.Constant()

freqs = np.linspace(1/Tspan,30/Tspan,30)

In [86]:
# # white noise
selection_ng = selections.Selection(selections.nanograv_backends)
selection_be = selections.Selection(selections.by_backend)

ef = white_signals.MeasurementNoise(efac=efac, selection=selection_be)
eq = white_signals.EquadNoise(log10_equad=equad, selection=selection_be)
ec = white_signals.EcorrKernelNoise(log10_ecorr=ecorr, selection=selection_ng)

# red noise (powerlaw with 30 frequencies)
pl = utils.powerlaw(log10_A=log10_A, gamma=gamma)
rn = gp_signals.FourierBasisGP(spectrum=pl, modes=freqs)#components=30,Tspan=Tspan)

# DM red noise (powerlaw with 30 frequencies)
dm_basis = utils.createfourierdesignmatrix_dm(nmodes=30)#freqs)
dm_pl = utils.powerlaw(log10_A=dm_log10_A, gamma=dm_gamma)
dm_gp = gp_signals.BasisGP(priorFunction=dm_pl, basisFunction=dm_basis, name='dm_gp')#components=30,Tspan=Tspan)

# gw (powerlaw with 10 frequencies)

gw_pl = utils.powerlaw(log10_A=gw_log10_A, gamma=gw_gamma)
gw = gp_signals.FourierBasisGP(spectrum=gw_pl, modes=freqs[:10], name='gw')#components=5, Tspan=Tspan, name='gw')

dm_dip = chromatic.dm_exponential_dip(tmin=48850.027042152935, tmax=57051.955677141625)
# timing model
tm = gp_signals.TimingModel()

In [87]:
model = tm + ef + eq + rn + dm_gp + gw 
model_ng = model + ec 

In [88]:
pnames = [p.name for p in psrs]


In [89]:
models = [model_ng(p) if 'NANOGrav' in p.flags['pta'] else model(p) for p in psrs]

In [90]:
dip_model = model_ng + dm_dip
J1713 = dip_model(psrs[pnames.index('J1713+0747')])
models[pnames.index('J1713+0747')] = J1713

In [91]:
pta = signal_base.PTA(models)

In [92]:
noise_path = '/Users/hazboun/ipta/classic_fixed/dr2_classic_noise.json'

In [93]:
with open(noise_path,'r') as fin:
    noise = json.load(fin)

In [94]:
pta.set_default_params(noise)

INFO: enterprise.signals.signal_base: Setting J0030+0451_430_ASP_efac to 1.1365193945368486
INFO: enterprise.signals.signal_base: Setting J0030+0451_430_PUPPI_efac to 0.9799469185765327
INFO: enterprise.signals.signal_base: Setting J0030+0451_EFF.EBPP.1360_efac to 1.1131774722050722
INFO: enterprise.signals.signal_base: Setting J0030+0451_EFF.EBPP.1410_efac to 0.7317297647944114
INFO: enterprise.signals.signal_base: Setting J0030+0451_EFF.EBPP.2639_efac to 0.9816169162003063
INFO: enterprise.signals.signal_base: Setting J0030+0451_JBO.DFB.1520_efac to 1.0323718949754754
INFO: enterprise.signals.signal_base: Setting J0030+0451_L-wide_ASP_efac to 1.1237984445609916
INFO: enterprise.signals.signal_base: Setting J0030+0451_L-wide_PUPPI_efac to 1.1395353301435847
INFO: enterprise.signals.signal_base: Setting J0030+0451_NRT.BON.1400_efac to 1.1918353704710902
INFO: enterprise.signals.signal_base: Setting J0030+0451_NRT.BON.1600_efac to 1.5756952974884049
INFO: enterprise.signals.signal_base:

INFO: enterprise.signals.signal_base: Setting J0437-4715_PDFB_40CM_log10_equad to -7.503948141950352
INFO: enterprise.signals.signal_base: Setting J0437-4715_PDFB_50CM_log10_equad to -6.597137744772705
INFO: enterprise.signals.signal_base: Setting J0437-4715_PKS.cpsr1.20cm_legacy_log10_equad to -6.956485100190424
INFO: enterprise.signals.signal_base: Setting J0437-4715_PKS.cpsr2.20cm_legacy_log10_equad to -8.023662046871959
INFO: enterprise.signals.signal_base: Setting J0437-4715_PKS.fptm.20cm-legacy_log10_equad to -6.425095378224767
INFO: enterprise.signals.signal_base: Setting J0437-4715_PKS.s2.20cm-legacy_log10_equad to -7.037802919641953
INFO: enterprise.signals.signal_base: Setting J0437-4715_WBCORR_10CM_log10_equad to -6.416938872758119
INFO: enterprise.signals.signal_base: Setting J0437-4715_WBCORR_20CM_log10_equad to -7.352602899412229
INFO: enterprise.signals.signal_base: Setting J0610-2100_JBO.DFB.1520_efac to 0.9561319405297859
INFO: enterprise.signals.signal_base: Setting J

INFO: enterprise.signals.signal_base: Setting J0621+1002_WSRT.P1.323.C_efac to 2.0634563476909142
INFO: enterprise.signals.signal_base: Setting J0621+1002_WSRT.P1.367.C_efac to 1.60570617347227
INFO: enterprise.signals.signal_base: Setting J0621+1002_EFF.EBPP.1360_log10_equad to -6.840229027469453
INFO: enterprise.signals.signal_base: Setting J0621+1002_EFF.EBPP.1410_log10_equad to -6.4136234859371495
INFO: enterprise.signals.signal_base: Setting J0621+1002_EFF.EBPP.2639_log10_equad to -6.741095969763628
INFO: enterprise.signals.signal_base: Setting J0621+1002_JBO.DFB.1400_log10_equad to -6.523470787480318
INFO: enterprise.signals.signal_base: Setting J0621+1002_JBO.DFB.1520_log10_equad to -7.046538643744288
INFO: enterprise.signals.signal_base: Setting J0621+1002_NRT.BON.1400_log10_equad to -7.084852706049046
INFO: enterprise.signals.signal_base: Setting J0621+1002_NRT.BON.1600_log10_equad to -6.789299166376719
INFO: enterprise.signals.signal_base: Setting J0621+1002_NRT.BON.2000_log1

INFO: enterprise.signals.signal_base: Setting J1012+5307_EFF.EBPP.2639_log10_equad to -7.1968778908834246
INFO: enterprise.signals.signal_base: Setting J1012+5307_JBO.DFB.1400_log10_equad to -7.31409009426702
INFO: enterprise.signals.signal_base: Setting J1012+5307_JBO.DFB.1520_log10_equad to -6.072720916928635
INFO: enterprise.signals.signal_base: Setting J1012+5307_NRT.BON.1400_log10_equad to -6.433160064221364
INFO: enterprise.signals.signal_base: Setting J1012+5307_NRT.BON.1600_log10_equad to -6.023914215850753
INFO: enterprise.signals.signal_base: Setting J1012+5307_NRT.BON.2000_log10_equad to -7.028164319799292
INFO: enterprise.signals.signal_base: Setting J1012+5307_Rcvr1_2_GASP_log10_equad to -7.420332253841796
INFO: enterprise.signals.signal_base: Setting J1012+5307_Rcvr1_2_GUPPI_log10_equad to -6.404566251351747
INFO: enterprise.signals.signal_base: Setting J1012+5307_Rcvr_800_GASP_log10_equad to -7.112374879793455
INFO: enterprise.signals.signal_base: Setting J1012+5307_Rcvr

INFO: enterprise.signals.signal_base: Setting J1600-3053_WBCORR_20CM_efac to 3.630038360475587
INFO: enterprise.signals.signal_base: Setting J1600-3053_CPSR2_20CM_log10_equad to -7.275261028391799
INFO: enterprise.signals.signal_base: Setting J1600-3053_CPSR2_50CM_log10_equad to -6.8712723879603255
INFO: enterprise.signals.signal_base: Setting J1600-3053_JBO.DFB.1520_log10_equad to -7.398181119937567
INFO: enterprise.signals.signal_base: Setting J1600-3053_NRT.BON.1400_log10_equad to -7.736119597740007
INFO: enterprise.signals.signal_base: Setting J1600-3053_NRT.BON.1600_log10_equad to -7.450061145742775
INFO: enterprise.signals.signal_base: Setting J1600-3053_NRT.BON.2000_log10_equad to -7.369477917696796
INFO: enterprise.signals.signal_base: Setting J1600-3053_PDFB_10CM_log10_equad to -6.493686179607709
INFO: enterprise.signals.signal_base: Setting J1600-3053_PDFB_20CM_log10_equad to -7.622415096598089
INFO: enterprise.signals.signal_base: Setting J1600-3053_PDFB_40CM_log10_equad to 

...

INFO: enterprise.signals.signal_base: Setting J1903+0327_L-wide_PUPPI_log10_equad to -7.592703350527445
INFO: enterprise.signals.signal_base: Setting J1903+0327_S-wide_ASP_log10_equad to -6.230007636310292
INFO: enterprise.signals.signal_base: Setting J1903+0327_S-wide_PUPPI_log10_equad to -7.51126913074776
INFO: enterprise.signals.signal_base: Setting J1903+0327_L-wide_ASP_log10_ecorr to -7.233065468324004
INFO: enterprise.signals.signal_base: Setting J1903+0327_L-wide_PUPPI_log10_ecorr to -5.957132167196923
INFO: enterprise.signals.signal_base: Setting J1903+0327_S-wide_ASP_log10_ecorr to -7.357913918666095
INFO: enterprise.signals.signal_base: Setting J1903+0327_S-wide_PUPPI_log10_ecorr to -7.19015845306828
INFO: enterprise.signals.signal_base: Setting J1909-3744_CPSR2_20CM_efac to 0.9957694403538463
INFO: enterprise.signals.signal_base: Setting J1909-3744_CPSR2_50CM_efac to 1.1219723851434589
INFO: enterprise.signals.signal_base: Setting J1909-3744_NRT.BON.1400_efac to 1.1798333674

INFO: enterprise.signals.signal_base: Setting J1918-0642_JBO.DFB.1520_log10_equad to -6.401437767836009
INFO: enterprise.signals.signal_base: Setting J1918-0642_NRT.BON.1400_log10_equad to -5.957594756748051
INFO: enterprise.signals.signal_base: Setting J1918-0642_NRT.BON.1600_log10_equad to -6.348413873661533
INFO: enterprise.signals.signal_base: Setting J1918-0642_Rcvr1_2_GASP_log10_equad to -7.654931517144057
INFO: enterprise.signals.signal_base: Setting J1918-0642_Rcvr1_2_GUPPI_log10_equad to -7.407176956710728
INFO: enterprise.signals.signal_base: Setting J1918-0642_Rcvr_800_GASP_log10_equad to -6.88473127071273
INFO: enterprise.signals.signal_base: Setting J1918-0642_Rcvr_800_GUPPI_log10_equad to -7.5859732528755695
INFO: enterprise.signals.signal_base: Setting J1918-0642_WSRT.P1.1380_log10_equad to -7.317444681228515
INFO: enterprise.signals.signal_base: Setting J1918-0642_WSRT.P1.1380.C_log10_equad to -7.033965106496656
INFO: enterprise.signals.signal_base: Setting J1918-0642_R

INFO: enterprise.signals.signal_base: Setting J1939+2134_S-wide_ASP_log10_ecorr to -6.704651353435267
INFO: enterprise.signals.signal_base: Setting J1939+2134_S-wide_PUPPI_log10_ecorr to -6.696621486648898
INFO: enterprise.signals.signal_base: Setting J1944+0907_430_ASP_efac to 1.1006635883881952
INFO: enterprise.signals.signal_base: Setting J1944+0907_430_PUPPI_efac to 1.2020082949244173
INFO: enterprise.signals.signal_base: Setting J1944+0907_L-wide_ASP_efac to 1.4053201642463713
INFO: enterprise.signals.signal_base: Setting J1944+0907_L-wide_PUPPI_efac to 1.3946160316471428
INFO: enterprise.signals.signal_base: Setting J1944+0907_430_ASP_log10_equad to -7.157736975311375
INFO: enterprise.signals.signal_base: Setting J1944+0907_430_PUPPI_log10_equad to -7.343291894241638
INFO: enterprise.signals.signal_base: Setting J1944+0907_L-wide_ASP_log10_equad to -7.287869228537538
INFO: enterprise.signals.signal_base: Setting J1944+0907_L-wide_PUPPI_log10_equad to -7.610918773791346
INFO: ente

INFO: enterprise.signals.signal_base: Setting J2124-3358_PKS.fptm.20cm-legacy_efac to 3.2499870266080624
INFO: enterprise.signals.signal_base: Setting J2124-3358_WBCORR_10CM_efac to 0.902080619696831
INFO: enterprise.signals.signal_base: Setting J2124-3358_WBCORR_20CM_efac to 1.083418876220534
INFO: enterprise.signals.signal_base: Setting J2124-3358_CPSR2_20CM_log10_equad to -7.12150770390982
INFO: enterprise.signals.signal_base: Setting J2124-3358_CPSR2_50CM_log10_equad to -6.707040282298372
INFO: enterprise.signals.signal_base: Setting J2124-3358_JBO.DFB.1400_log10_equad to -6.791797019607793
INFO: enterprise.signals.signal_base: Setting J2124-3358_JBO.DFB.1520_log10_equad to -6.6953406173648755
INFO: enterprise.signals.signal_base: Setting J2124-3358_NRT.BON.1400_log10_equad to -7.2437178075168
INFO: enterprise.signals.signal_base: Setting J2124-3358_NRT.BON.1600_log10_equad to -7.192639108571274
INFO: enterprise.signals.signal_base: Setting J2124-3358_NRT.BON.2000_log10_equad to -5

INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr_800_GUPPI_log10_equad to -6.320952521279583
INFO: enterprise.signals.signal_base: Setting J2145-0750_WBCORR_10CM_log10_equad to -6.929935148833157
INFO: enterprise.signals.signal_base: Setting J2145-0750_WBCORR_20CM_log10_equad to -5.7901537804908285
INFO: enterprise.signals.signal_base: Setting J2145-0750_WSRT.P1.1380.C_log10_equad to -7.226292155337077
INFO: enterprise.signals.signal_base: Setting J2145-0750_WSRT.P1.2273.C_log10_equad to -6.725515164016669
INFO: enterprise.signals.signal_base: Setting J2145-0750_WSRT.P1.323.C_log10_equad to -6.624669363296452
INFO: enterprise.signals.signal_base: Setting J2145-0750_WSRT.P1.367.C_log10_equad to -5.3435965806883345
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr1_2_GASP_log10_ecorr to -5.7393856426104275
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr1_2_GUPPI_log10_ecorr to -6.398455780034647
INFO: enterprise.signals.signal_base: Setting J2145-07

In [95]:
nihan = co.Core(label='Nihan\'s chains',
                chaindir='./classic/model_2a_fg/')
cH = co.HyperModelCore(label='Jeff Hypermodel', 
                       chaindir='./classic/model2a_vs_3a/10freqs/fixed_gamma/')
jeff = cH.model_core(0)

Appending PTMCMCSampler sampling parameters to end of parameter list.
If unwanted please provide a parameter list.
No burn specified. Burn set to 25% of chain length, 20475
You may change the burn length with core.set_burn()
Red noise frequencies must be set before plotting most red noise figures.
Please use core.set_rn_freqs() to set, if needed.
Appending PTMCMCSampler sampling parameters to end of parameter list.
If unwanted please provide a parameter list.
No burn specified. Burn set to 25% of chain length, 31325
You may change the burn length with core.set_burn()
Red noise frequencies must be set before plotting most red noise figures.
Please use core.set_rn_freqs() to set, if needed.


In [96]:
for idx in np.random.randint(jeff.burn, jeff.chain.shape[0],20):
    x0 = jeff.chain[idx,:-4]
#     print(x0)
    lnlike = pta.get_lnlikelihood({nm:p for nm, p in zip(pta.param_names,x0)})
    lnprior = pta.get_lnprior({nm:p for nm, p in zip(pta.param_names,x0)})
    print(lnlike,jeff.chain[idx,-3])
    print(lnlike+lnprior,jeff.chain[idx,-4])
    print('-----')

2236441.38696982 2236441.387167
2235988.820537583 2235536.254303
-----
2236452.9611875764 2236452.961395
2236000.3947553392 2235547.82853
-----
2236438.75801288 2236438.758199
2235986.191580643 2235533.625334
-----
2236438.5767831975 2236438.577001
2235986.0103509603 2235533.444137
-----
2236443.847910798 2236443.848072
2235991.281478561 2235538.715207
-----
2236433.5509863673 2236433.551176
2235980.98455413 2235528.418312
-----
2236439.146074362 2236439.14632
2235986.579642125 2235534.013455
-----
2236452.5123050846 2236452.512588
2235999.9458728475 2235547.379723
-----
2236467.050424175 2236467.050588
2236014.4839919377 2235561.917724
-----
2236441.328776373 2236441.328966
2235988.762344136 2235536.196102
-----
2236429.7917908463 2236429.792014
2235977.225358609 2235524.659149
-----
2236456.4624381056 2236456.462625
2236003.8960058684 2235551.329761
-----
2236431.83307753 2236431.833295
2235979.2666452928 2235526.70043
-----
2236427.1681420584 2236427.168329
2235974.6017098213 223552

In [97]:
for idx in np.random.randint(nihan.burn, nihan.chain.shape[0],20):
    x0 = nihan.chain[idx,:-4]
#     print(x0)
    lnlike = pta.get_lnlikelihood({nm:p for nm, p in zip(pta.param_names,x0)})
    lnprior = pta.get_lnprior({nm:p for nm, p in zip(pta.param_names,x0)})
    print(lnprior)
    print(lnlike,nihan.chain[idx,-3])
    print(lnlike+lnprior,nihan.chain[idx,-4])
    print('-----')

-452.5664322371614
2236456.007438715 2236456.007439
2236003.441006478 2236003.441007
-----
-452.5664322371614
2236437.8949454366 2236437.894946
2235985.3285131995 2235985.328514
-----
-452.5664322371614
2236436.118338384 2236436.118339
2235983.551906147 2235983.551907
-----
-452.5664322371614
2236446.2444239105 2236446.244424
2235993.6779916734 2235993.677992
-----
-452.5664322371614
2236453.0268351585 2236453.026835
2236000.4604029213 2236000.460403
-----
-452.5664322371614
2236449.7846285705 2236449.784629
2235997.2181963334 2235997.218197
-----
-452.5664322371614
2236445.2213777136 2236445.221378
2235992.6549454764 2235992.654946
-----
-452.5664322371614
2236414.9413542044 2236414.941355
2235962.3749219673 2235962.374922
-----
-452.5664322371614
2236454.5480603604 2236454.548061
2236001.981628123 2236001.981629
-----
-452.5664322371614
2236450.860489796 2236450.86049
2235998.2940575588 2235998.294058
-----
-452.5664322371614
2236444.233770886 2236444.233771
2235991.667338649 2235991

In [98]:
pta.get_lnlikelihood({nm:p.sample() for nm, p in zip(pta.param_names,pta.params)})

2149021.0501645836

In [102]:
hpta=hypermodel.HyperModel({0:pta,1:pta,2:pta})

In [103]:
for idx in np.random.randint(jeff.burn, jeff.chain.shape[0],20):
    x0 = jeff.chain[idx,:-4]
#     print(x0)
    y0 = np.append(x0,0)
    lnlike = pta.get_lnlikelihood({nm:p for nm, p in zip(pta.param_names,x0)})
    lnprior = pta.get_lnprior({nm:p for nm, p in zip(pta.param_names,x0)})
    hlnlike = hpta.get_lnlikelihood(y0)
    hlnprior = hpta.get_lnprior(y0)
    print(lnlike,jeff.chain[idx,-3])
    print(hlnlike,jeff.chain[idx,-3])
    print(lnlike+lnprior,jeff.chain[idx,-4])
    print(hlnlike+hlnprior,jeff.chain[idx,-4])
    print(lnprior,jeff.chain[idx,-4]-jeff.chain[idx,-3])
    print(hlnprior,jeff.chain[idx,-4]-jeff.chain[idx,-3])
    print('-----')

2236441.730287509 2236441.730473
2236441.730287509 2236441.730473
2235989.163855272 2235536.597609
2235084.0309907976 2235536.597609
-452.5664322371614 -905.1328640002757
-1357.6992967114843 -905.1328640002757
-----
2236456.3271649107 2236456.327406
2236456.3271649107 2236456.327406
2236003.7607326736 2235551.194541
2235098.6278681993 2235551.194541
-452.5664322371614 -905.1328650000505
-1357.6992967114843 -905.1328650000505
-----
2236425.123845009 2236425.12401
2236425.123845009 2236425.12401
2235972.557412772 2235519.991146
2235067.4245482977 2235519.991146
-452.5664322371614 -905.1328639998101
-1357.6992967114843 -905.1328639998101
-----
2236427.905865195 2236427.906097
2236427.905865195 2236427.906097
2235975.339432958 2235522.773232
2235070.2065684837 2235522.773232
-452.5664322371614 -905.1328650000505
-1357.6992967114843 -905.1328650000505
-----
2236436.170562119 2236436.170721
2236436.170562119 2236436.170721
2235983.604129882 2235531.037856
2235078.4712654077 2235531.037856
-4

In [101]:
for idx in np.random.randint(nihan.burn, nihan.chain.shape[0],20):
    x0 = nihan.chain[idx,:-4]
#     print(x0)
    y0 = np.append(x0,0)
    lnlike = pta.get_lnlikelihood({nm:p for nm, p in zip(pta.param_names,x0)})
    lnprior = pta.get_lnprior({nm:p for nm, p in zip(pta.param_names,x0)})
    hlnlike = hpta.get_lnlikelihood(y0)
    hlnprior = hpta.get_lnprior(y0)
    print(lnlike,nihan.chain[idx,-3])
    print(hlnlike,nihan.chain[idx,-3])
    print(lnlike+lnprior,nihan.chain[idx,-4])
    print(hlnlike+hlnprior,nihan.chain[idx,-4])
    print(lnprior,nihan.chain[idx,-4]-nihan.chain[idx,-3])
    print(hlnprior,nihan.chain[idx,-4]-nihan.chain[idx,-3])
    print('-----')

2236447.9963046415 2236447.996305
2236447.9963046415 2236447.996305
2235995.4298724043 2235995.429873
2235542.863440167 2235995.429873
-452.5664322371614 -452.56643200013787
-905.1328644743228 -452.56643200013787
-----
2236436.087128474 2236436.087129
2236436.087128474 2236436.087129
2235983.5206962368 2235983.520696
2235530.9542639996 2235983.520696
-452.5664322371614 -452.56643299991265
-905.1328644743228 -452.56643299991265
-----
2236431.295518312 2236431.295519
2236431.295518312 2236431.295519
2235978.729086075 2235978.729087
2235526.162653838 2235978.729087
-452.5664322371614 -452.56643200013787
-905.1328644743228 -452.56643200013787
-----
2236433.4199841167 2236433.419985
2236433.4199841167 2236433.419985
2235980.8535518795 2235980.853552
2235528.2871196424 2235980.853552
-452.5664322371614 -452.56643299991265
-905.1328644743228 -452.56643299991265
-----
2236431.338252748 2236431.338253
2236431.338252748 2236431.338253
2235978.7718205107 2235978.771821
2235526.2053882736 2235978.

In [69]:
np.rint(0.1)

0.0

In [75]:
hpta.get_lnlikelihood(y0)

2235837.5179618923